In [5]:
import numpy as np
import pickle
import time
import sys
import cv2
import math
from scipy.signal import medfilt
from scipy.ndimage.filters import maximum_filter as maxfilt

In [6]:
def get_chunk(start_x, start_y, end_x, end_y, img, size):
    chunk = np.zeros(size)
    chunk_x = chunk_y = -1
    #print(start_x, start_y, end_x, end_y)
    for i in range(start_x, end_x):
        chunk_x += 1
        chunk_y = -1
        for j in range(start_y, end_y):
            chunk_y += 1
            #print(chunk_x, chunk_y, i, j)
            if (0 <= i < len(img)) and (0 <= j < len(img[0])) :
                chunk[chunk_x, chunk_y] = img[i,j]
    return chunk
    

def calculate_rank_matrix(img):
    diff = 2
    final_arr = np.zeros(img.shape)
    for cur_x in range(img.shape[0]):
        for cur_y in range(img.shape[1]):
            start_x, start_y = cur_x - diff, cur_y - diff
            end_x, end_y = cur_x + diff+1, cur_y + diff+1

            if (start_x < 0) or\
                (start_y < 0) or\
                (end_x > img.shape[0]) or\
                (end_y > img.shape[1]):
                chunk = get_chunk(start_x, start_y, end_x, end_y, img, [5,5])
            else:    
                chunk = img[start_x :end_x, start_y: end_y]
            chunk = chunk > chunk[2, 2]
            final_arr[cur_x, cur_y] = np.concatenate(chunk).sum()
    return final_arr
    
def calculate_disp_matrix(rank_l, rank_r, diff, disp_range):
    if diff == 1:
        size = [3,3]
    if diff == 7:
        size = [15,15]
    
    final_arr = np.zeros(rank_l.shape)
    for cur_x in range(rank_l.shape[0]):      
        if (cur_x % 25) == 0:
            print("%s/%s" % (cur_x, rank_l.shape[0]), end="\t")
            
        for cur_y in range(rank_l.shape[1]):
            start_x, start_y = cur_x - diff, cur_y - diff
            end_x, end_y = cur_x + diff+1, cur_y + diff+1

            if (start_x < 0) or\
                (start_y < 0) or\
                (end_x > rank_l.shape[0]) or\
                (end_y > rank_l.shape[1]):
                chunk = get_chunk(start_x, start_y, end_x, end_y, rank_l, size)
            else:    
                chunk = rank_l[start_x :end_x, start_y: end_y]
            min_abs = float("inf")
            disparity = 0
            for i in range(disp_range):
                new_start_y, new_end_y = start_y + i, end_y + i
                
                if (start_x < 0) or\
                (new_start_y < 0) or\
                (end_x > rank_r.shape[0]) or\
                (new_end_y > rank_r.shape[1]):
                    new_chunk = get_chunk(start_x, new_start_y, end_x, new_end_y, rank_r, size)
                else:
                    new_chunk = rank_r[start_x :end_x, new_start_y: new_end_y]
                
                result = np.absolute(np.array(chunk) - np.array(new_chunk))
                result = result.sum()

                if result < min_abs:
                    min_abs = result
                    disparity = i

            final_arr[cur_x, cur_y] =  disparity 
                
            
    return final_arr

def calculate_error(true_disp, calc_disp):
    diff_matrix = true_disp/4 - calc_disp
    diff_matrix = np.absolute(np.round(diff_matrix))
    error = (diff_matrix[diff_matrix > 1].size / true_disp.size) * 100
    return error

def write_image(title,img):
    cv2.imwrite(".\\Output_Images\\%s.jpeg" % title, img)
    


In [13]:
def PointCloud2Image(M,Sets3DRGB,viewport,filter_size):

    # setting yp output image
    print("...Initializing 2D image...")
    top = viewport[0]
    left = viewport[1]
    h = viewport[2]
    w = viewport[3]
    bot = top  + h + 1
    right = left + w +1;
    output_image = np.zeros((h+1,w+1,3));    

    for counter in range(len(Sets3DRGB)):
        print("...Projecting point cloud into image plane...")

        # clear drawing area of current layer
        canvas = np.zeros((bot,right,3))

        # segregate 3D points from color
        dataset = Sets3DRGB[counter]
        P3D = dataset[:3,:]
        color = (dataset[3:6,:]).T
        
        # form homogeneous 3D points (4xN)
        len_P = len(P3D[1])
        ones = np.ones((1,len_P))
        X = np.concatenate((P3D, ones))
        
        
        # apply (3x4) projection matrix
  
        x = np.matmul(M,X)
        
        # normalize by 3rd homogeneous coordinate
        x = np.around(np.divide(x, np.array([x[2,:],x[2,:],x[2,:]])))

        # truncate image coordinates
        x[:2,:] = np.floor(x[:2,:])

        # determine indices to image points within crop area
        i1 = x[1,:] > top
        i2 = x[0,:] > left
        i3 = x[1,:] < bot
        i4 = x[0,:] < right
        ix = np.logical_and(i1, np.logical_and(i2, np.logical_and(i3, i4)))

        # make reduced copies of image points and cooresponding color
        rx = x[:,ix]
        rcolor = color[ix,:]

        for i in range(len(rx[0])):
            canvas[int(rx[1,i]),int(rx[0,i]),:] = rcolor[i,:]

        # crop canvas to desired output size
        cropped_canvas = canvas[top:top+h+1,left:left+w+1]

        # filter individual color channels
        shape = cropped_canvas.shape
        
    
        filtered_cropped_canvas = cropped_canvas

        
        # get indices of pixel drawn in the current canvas
        drawn_pixels = np.sum(filtered_cropped_canvas,2)
        idx = drawn_pixels != 0
        shape = idx.shape
        shape = (shape[0],shape[1],3)
        idxx = np.zeros(shape,dtype=bool)

        # make a 3-channel copy of the indices
        idxx[:,:,0] = idx
        idxx[:,:,1] = idx
        idxx[:,:,2] = idx

        # erase canvas drawn pixels from the output image
        output_image[idxx] = 0

        #sum current canvas on top of output image
        output_image = output_image + filtered_cropped_canvas

    print("Done")
    return output_image



def main():
    file_p = open("Input_data/data.obj",'rb')
    camera_objs = pickle.load(file_p)

    # extract objects from object array
    crop_region = camera_objs[0].flatten()
    
    filter_size = camera_objs[1].flatten()
    K = camera_objs[2]
    ForegroundPointCloudRGB = camera_objs[3]
    BackgroundPointCloudRGB = camera_objs[4]
    
    t = np.array([0, 0, 0]).reshape((3,1)).astype(float) #Initially translation kept as 0
    R = np.identity(3) #Rotation kept as identity
    
    # create variables for computation
    data3DC = (BackgroundPointCloudRGB,ForegroundPointCloudRGB)    
    
    ###### Dividing crop region and internal camera parameter with 4 to reduce the size of the image #####
    crop_region = (crop_region/4).astype(int)
    K[0,0] /= 4 #Fx 
    K[1,1] /= 4 #Fy
    K[0,2] /= 4 #Cx
    K[1,2] /= 4 #Cy
    
    print("New crop_region", crop_region)
    print("New K", K)
    
    #FOR LOOP iterating over translation to generate sterio images.
    val = [-0.1, 0.1] #Translations
    for idx in range(2):
        image_dir = ".\\Output_Images\\"
        fname = image_dir + "Sterio_Image_{}.jpg".format(idx)
        print("\nGenerating {}".format(fname))

        t[0] = val[idx] #Translation in X direction added

        M = np.matmul(K,(np.hstack((R,t))))
        img = PointCloud2Image(M,data3DC,crop_region,filter_size)
        
        # Convert image values form (0-1) to (0-255) and cahnge type from float64 to float32
        img = 255*(np.array(img, dtype=np.float32))
        
        # convert image from RGB to BGR for OpenCV
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
   
        cv2.imwrite(fname, img_bgr)


In [14]:
main()

img_L = cv2.imread('.//Output_Images/Sterio_Image_1.jpg')
img_R = cv2.imread('.//Output_Images/Sterio_Image_0.jpg')

grey_L = np.array(img_L[:,:,0]*0.11 + img_L[:,:,1]*0.3 + img_L[:,:,2]*0.59).reshape((img_L.shape[0], img_L.shape[1], 1))

grey_R = np.array(img_R[:,:,0]*0.11 + img_R[:,:,1]*0.3 + img_R[:,:,2]*0.59).reshape((img_R.shape[0], img_R.shape[1], 1))

rank_transformed_L = calculate_rank_matrix(grey_L)
rank_transformed_R = calculate_rank_matrix(grey_R)

print("\n\nCalculating disparity map for 15x15 window:")
disp_img_15 =  calculate_disp_matrix(rank_transformed_R, rank_transformed_L, 7, 40)


write_image("Sterio_Disparity", disp_img_15.astype("uint8"))
disp_img_15 *= 4
write_image("Sterio_Disparity_Amplified", disp_img_15.astype("uint8"))

New crop_region [  0   0 512 768]
New K [[689.875   0.    380.175]
 [  0.    691.05  251.7  ]
 [  0.      0.      1.   ]]

Generating .\Output_Images\Sterio_Image_0.jpg
...Initializing 2D image...
...Projecting point cloud into image plane...
...Projecting point cloud into image plane...
Done

Generating .\Output_Images\Sterio_Image_1.jpg
...Initializing 2D image...
...Projecting point cloud into image plane...
...Projecting point cloud into image plane...
Done


Calculating disparity map for 15x15 window:
0/513	25/513	50/513	75/513	100/513	125/513	150/513	175/513	200/513	225/513	250/513	275/513	300/513	325/513	350/513	375/513	400/513	425/513	450/513	475/513	500/513	